In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

In [177]:
df = pd.read_csv(r"C:\Users\PC\Desktop\Bengaluru_House_Data.csv")
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [178]:
df.drop(columns = ['availability', 'society', 'balcony'], inplace = True)
df.head()

,area_type,location,size,total_sqft,bath,price
0,Super built-up Area,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Plot Area,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Built-up Area,Uttarahalli,3 BHK,1440,2.0,62.00
3,Super built-up Area,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Super built-up Area,Kothanur,2 BHK,1200,2.0,51.00


In [179]:
df['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', nan, '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [180]:
df['bedroom'] = df['size'].str.split(' ', expand = True)[0]

In [181]:
df.dropna(inplace = True)

In [182]:
df['bedroom'] = df['bedroom'].astype(int)

In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13246 entries, 0 to 13319
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   area_type   13246 non-null  object 
 1   location    13246 non-null  object 
 2   size        13246 non-null  object 
 3   total_sqft  13246 non-null  object 
 4   bath        13246 non-null  float64
 5   price       13246 non-null  float64
 6   bedroom     13246 non-null  int32  
dtypes: float64(2), int32(1), object(4)
memory usage: 776.1+ KB


In [184]:
df.drop('size', axis = 'columns', inplace = True)

In [185]:
df[df['bedroom'] > 15]

,area_type,location,total_sqft,bath,price,bedroom
1718,Super built-up Area,2Electronic City Phase II,8000,27.0,230.0,27
3379,Super built-up Area,1Hanuman Nagar,2000,16.0,490.0,19
3609,Super built-up Area,Koramangala Industrial Layout,10000,16.0,550.0,16
4684,Plot Area,Munnekollal,2400,40.0,660.0,43
11559,Plot Area,1Kasavanhalli,1200,18.0,200.0,18


In [186]:
def normalize(x):
    char = x.split('-')
    if len(char) == 2:
        return (float(char[0]) + float(char[1]))/2
    try:
        return float(x)
    except:
        return None

In [187]:
df['total_sqft'] = df['total_sqft'].apply(normalize)

In [188]:
df.dropna(inplace = True)

In [189]:
df1 = df.copy()
df1.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13200 entries, 0 to 13319
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   area_type   13200 non-null  object 
 1   location    13200 non-null  object 
 2   total_sqft  13200 non-null  float64
 3   bath        13200 non-null  float64
 4   price       13200 non-null  float64
 5   bedroom     13200 non-null  int32  
dtypes: float64(3), int32(1), object(2)
memory usage: 670.3+ KB


In [190]:
loc_list = df1.groupby('location')['location'].count().sort_values(ascending = False)
loc_less_than_11 = loc_list[loc_list <= 10]

In [191]:
df1['location'] = df1['location'].apply(lambda x: 'others' if x in loc_less_than_11 else x)
df1['location'].value_counts()

others                       2887
Whitefield                    532
Sarjapur  Road                392
Electronic City               302
Kanakpura Road                264
                             ... 
Pattandur Agrahara             11
2nd Phase Judicial Layout      11
Bommenahalli                   11
Marsur                         11
LB Shastri Nagar               11
Name: location, Length: 241, dtype: int64

In [205]:
df1['price_per_sqft'] = (df['price']*100000/df['total_sqft']).round()
df2 = df1[~(df1['total_sqft']/df1['bedroom'] < 300)]

In [206]:
df2['price_per_sqft'].describe()

count     12456.000000
mean       6308.499759
std        4168.132109
min         268.000000
25%        4211.000000
50%        5294.000000
75%        6917.000000
max      176471.000000
Name: price_per_sqft, dtype: float64

In [215]:
from pprint import pprint

In [232]:
def remove_outliers(df):
    final_df = pd.DataFrame()
    for key, subdf in df.groupby('location'):

        mean = subdf['price_per_sqft'].mean()
        std = subdf['price_per_sqft'].std()

        reduced_subdf = subdf[(subdf['price_per_sqft'] >= mean - std) & (subdf['price_per_sqft'] <= mean + std)]
        final_df = pd.concat([final_df, reduced_subdf], ignore_index = True)
        
    return final_df

In [233]:
df2 = remove_outliers(df1)
df2.shape

(11485, 7)

In [234]:
df2.head()

,area_type,location,total_sqft,bath,price,bedroom,price_per_sqft
0,Super built-up Area,Devarachikkanahalli,1250.0,2.0,44.0,3,3520.0
1,Super built-up Area,Devarachikkanahalli,1250.0,2.0,40.0,2,3200.0
2,Plot Area,Devarachikkanahalli,1200.0,2.0,83.0,2,6917.0
3,Super built-up Area,Devarachikkanahalli,1170.0,2.0,40.0,2,3419.0
4,Super built-up Area,Devarachikkanahalli,1425.0,2.0,65.0,3,4561.0


In [ ]:
def remove_bedroom_outliers():
    